In [1]:
import CG.x64.Debug.CG as CG
import unittest
import numpy as np
import time
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d
output_notebook()

Loading BokehJS ...

In [2]:
def gen_points(m: int) -> CG.Point2DVector:
    x = np.random.rand(m)
    y = np.random.rand(m)
    res = CG.Point2DVector()
    for i in range(len(x)):
        res.append(CG.Point2D(x[i], y[i]))
    return res

In [3]:
from cProfile import label
from turtle import fillcolor


def draw_points_2d(points: CG.Point2DVector):
    f = figure()
    x = [p.x() for p in points]
    y = [p.y() for p in points]
    f.scatter(x, y)
    show(f)
    
def draw_line_2d(points: CG.Point2DVector):
    f = figure()
    x = [p.x() for p in points]
    y = [p.y() for p in points]
    text = [str(i) for i in range(len(x))]
    source = ColumnDataSource(data=dict(x=x, y=y, text=text))
    labels = LabelSet(x='x', y='y', text='text', text_font_size='10px', source=source, 
                      x_offset=3, y_offset=3, render_mode='canvas')
    f.scatter(x='x',y='y',source=source)
    f.line(x, y, line_dash=[4, 4], line_color="orange", line_width=1)
    f.add_layout(labels)
    show(f)
    
def draw_convex_result(ori: CG.Point2DVector, extrem_pts: CG.Point2DVector):
    f = figure()
    x_ori = [p.x() for p in ori]
    y_ori = [p.y() for p in ori]
    f.scatter(x_ori, y_ori)
    x_extrem = [p.x() for p in extrem_pts]
    y_extrem = [p.y() for p in extrem_pts]
    text = [str(i) for i in range(len(x_extrem))]
    source = ColumnDataSource(data=dict(x=x_extrem, y=y_extrem, text=text))
    labels = LabelSet(x='x', y='y', text='text', text_font_size='10px', source=source, 
                      x_offset=3, y_offset=3, render_mode='canvas')
    f.scatter(x='x',y='y',source=source)
    f.line(x_extrem, y_extrem, line_dash=[4, 4], line_color="red", line_width=1)
    f.add_layout(labels)
    show(f)

In [4]:
class TestGeomertry(unittest.TestCase):
    def test_in_triangle1(self):
        # 顺时针排序
        p = CG.Point2D(5.0, 0.0)
        q = CG.Point2D(0.0, 0.0)
        r = CG.Point2D(0.0, 5.0)
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(0.0, 0.0)))
        self.assertTrue(CG.in_triangle(p, q, r, CG.Point2D(1.0, 1.0)))
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(5.0, 5.0))) 
    def test_in_triangle2(self):
        # 逆时针排序
        p = CG.Point2D(5.0, 0.0)
        q = CG.Point2D(0.0, 5.0)
        r = CG.Point2D(0.0, 0.0)
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(0.0, 0.0)))
        self.assertTrue(CG.in_triangle(p, q, r, CG.Point2D(1.0, 1.0)))
        self.assertFalse(CG.in_triangle(p, q, r, CG.Point2D(5.0, 5.0)))
    def test_in_convex_polygon(self):
        convex_hull = CG.Point2DVector([
            CG.Point2D(0.0, 0.0),
            CG.Point2D(1.0, 0.0),
            CG.Point2D(2.0, 1.0),
            CG.Point2D(1.0, 2.0),
            CG.Point2D(0.0, 1.0)])
        self.assertFalse(CG.in_convex_polygon(convex_hull, CG.Point2D(0.0, 0.0)))
        self.assertTrue(CG.in_convex_polygon(convex_hull, CG.Point2D(0.5, 0.5)))
    def test_sort_by_angle(self):
        pts = gen_points(10)
        draw_line_2d(pts)
        ordered_pts = CG.sort_by_angle(CG.Point2D(0.0, 0.0), pts)
        draw_line_2d(ordered_pts)

In [5]:
def test_geometry():
    suite = unittest.TestSuite()
    suite.addTest(TestGeomertry('test_in_triangle1'))
    suite.addTest(TestGeomertry('test_in_triangle2'))
    suite.addTest(TestGeomertry('test_in_convex_polygon'))
    suite.addTest(TestGeomertry('test_sort_by_angle'))
    return suite
runner = unittest.TextTestRunner()
runner.run(test_geometry())

...

.
----------------------------------------------------------------------
Ran 4 tests in 0.087s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [6]:
class TestConvexHull(unittest.TestCase):
    def __init__(self, methodName):
        super(TestConvexHull, self).__init__(methodName)
        self.polygon_points_ = gen_points(10000)
        
    def test_ExtremEdgeAlgo(self):
        algo = CG.ExtremEdgeAlgo()
        t = time.perf_counter()
        res = algo.gen_extrem_points(self.polygon_points_)
        print("ExtremEdgeAlgo time cost {}".format(time.perf_counter() - t))
        draw_convex_result(self.polygon_points_, res)
        
    def test_IncrementalAlgo(self):
        algo = CG.IncrementalAlgo()
        t = time.perf_counter()
        res = algo.gen_extrem_points(self.polygon_points_)
        print("IncrementalAlgo time cost {}".format(time.perf_counter() - t))
        draw_convex_result(self.polygon_points_, res)    
        
    def test_JarvisMarchAlgo(self):
        algo = CG.JarvisMarchAlgo()
        t = time.perf_counter()
        res = algo.gen_extrem_points(self.polygon_points_)
        print("JarvisMarchAlgo time cost {}".format(time.perf_counter() - t))
        draw_convex_result(self.polygon_points_, res)   
        
    def test_GrahamScaneAlgo(self):
        algo = CG.GrahamScaneAlgo()
        t = time.perf_counter()
        res = algo.gen_extrem_points(self.polygon_points_)
        print("GrahamScaneAlgo time cost {}".format(time.perf_counter() - t))
        draw_convex_result(self.polygon_points_, res)         

In [7]:
def test_convex_hull():
    suite = unittest.TestSuite()
    suite.addTest(TestConvexHull('test_ExtremEdgeAlgo'))
    suite.addTest(TestConvexHull('test_IncrementalAlgo'))
    suite.addTest(TestConvexHull('test_JarvisMarchAlgo'))
    suite.addTest(TestConvexHull('test_GrahamScaneAlgo'))
    return suite
runner = unittest.TextTestRunner()
runner.run(test_convex_hull())

ExtremEdgeAlgo time cost 0.9329394000000022


.

IncrementalAlgo time cost 0.037205399999997724


.

JarvisMarchAlgo time cost 0.01784059999999954


.

GrahamScaneAlgo time cost 0.04853450000000237


.
----------------------------------------------------------------------
Ran 4 tests in 4.828s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>